In [2]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document 
from langchain.vectorstores.chroma import Chroma 
from dotenv import load_dotenv 
from langchain.chat_models import ChatOpenAI 
import os 
import shutil 
from docx import Document
from pptx import Presentation
import numpy as np

In [3]:
os.environ['OPENAI_API_KEY'] = ('<open-ai_api_key>')

## Methods

In [3]:
FOLDER_PATH = './Chroma_Data/'
from langchain_community.document_loaders import PyPDFLoader

In [4]:
def read_word_file(file_path):
    doc = Document(os.path.join(FOLDER_PATH, file_path))
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)

In [5]:
# Function to read a PowerPoint file
def read_pptx_file(file_path):
    prs = Presentation(os.path.join(FOLDER_PATH, file_path))
    text_runs = []
    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text_runs.append(run.text)
    return '\n'.join(text_runs)

In [6]:
def read_pdf_file(file_path):
    
    loader = PyPDFLoader(os.path.join(FOLDER_PATH, file_path))
    pages = loader.load_and_split()

    return_text = ""
    for page in pages:
        return_text += page.page_content + "\n\n"
    
    return return_text

In [7]:
os.listdir('./Chroma_Data/')

['Logos Avans Prosedürü V2.docx',
 'Logos Avans Prosedürü.docx',
 'Logos ERP Ticket Prosedürü.pdf',
 'Logos Seyahat Prosedürü Sunumu.pptx',
 'Logos_Temsil Ağırlama Prosedürü.pptx',
 'Tekstil Şirketi Acil Yardım Prosedürü.docx']

In [8]:
read_docs = []

for doc_file_name in os.listdir('./Chroma_Data/'):
    if 'pdf' in doc_file_name:
        data = read_pdf_file(doc_file_name)
    elif 'pptx' in doc_file_name:
        data = read_pptx_file(doc_file_name)
    elif 'docx' in doc_file_name:
        data = read_word_file(doc_file_name)

    
    else:
        raise ValueError(f"{doc_file_name} could not found.")
    print(data)
    read_docs.append(data)
    

Logos Şirketi Avans Prosedürü
1. Amaç: Bu prosedür, Logos Şirketi çalışanlarının acil nakit ihtiyaçlarını karşılamak amacıyla avans taleplerini düzenlemeyi ve yönetmeyi amaçlar. Prosedür, çeşitli durumlar için avans türlerini ve ödeme planlarını belirler.
2. Kapsam: Bu prosedür, Logos Şirketi bünyesinde tam zamanlı olarak çalışan tüm personeli kapsar. Yarı zamanlı, stajyer ve geçici personel bu prosedür kapsamında değildir.
3. Tanımlar:
Avans: Çalışanların maaşlarından kesilmek üzere önden verilen nakit paradır.
İK Müdürü: Avans taleplerini inceleyip onaylayan ve süreci yöneten İnsan Kaynakları müdürüdür.
Muhasebe Departmanı: Onaylanan avans taleplerinin ödemesini gerçekleştiren departmandır.
4. Avans Türleri:
Standart Avans: Acil nakit ihtiyaçları için verilen avanstır.
Doğum, Ölüm, Evlilik ve Sağlık Avansı: Doğum, ölüm, evlilik ve 1. dereceden yakın (anne-baba) sağlık durumları için verilen ve taksitlendirilebilir avanstır.
İş Avansı: İş gereği yapılan harcamalar için verilen avanstı

In [9]:
np.shape(read_docs)

(6,)

In [10]:
read_docs[-1]

'Tekstil Şirketi Acil Yardım Prosedürü\n1. Acil Durum Tanımı ve Türleri\nAcil durumlar, işletme faaliyetlerini kesintiye uğratan veya çalışanların, ziyaretçilerin ya da çevrenin güvenliğini tehdit eden beklenmedik olaylardır. Tekstil sektöründe karşılaşılabilecek acil durum türleri şunlardır:\nYangın\nKimyasal dökülme veya sızıntı\nMakine kazaları\nDoğal afetler (deprem, sel, fırtına)\nSağlık acil durumları (bayılma, ciddi yaralanma)\n2. Acil Durum İletişimi\nAcil Durum İhbarı: Herhangi bir acil durum fark edildiğinde, derhal acil durum telefon numaraları aranmalıdır. Şirketin iç hat numaraları ve acil durum numaraları, her çalışma alanında görülebilecek şekilde asılmalıdır.\nAcil Bildirim Sistemi: Şirket içi dijital platform (örneğin, MUÇO sistemi) üzerinden acil durum bildirimi yapılabilir. Acil bildirim modülü, "Emergency Notification! I\'m Not Safe!" gibi hızlı bildirim seçeneklerini içermelidir.\n3. Acil Durum Ekipleri ve Görevleri\nAcil Durum Koordinatörü: Acil durum yönetiminden

In [11]:
def split_text(documents: list):
    """
    Split the text content of the given list of Document objects into smaller chunks.
    Args:
    documents (list[Document]): List of Document objects containing text content to split.
    Returns:
    list[Document]: List of Document objects representing the split text chunks.
    """
    # Initialize text splitter with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, # Size of each chunk in characters
    chunk_overlap=200, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
    )
    
    # Split documents into smaller chunks using text splitter
    chunks = []
    for document in documents:
        chunks.extend(text_splitter.split_text(document)) 
    
    
    return chunks # Return the list of split text chunks
    
    
chunks = split_text(read_docs)

In [12]:
np.shape(chunks)

(132,)

In [13]:
# Path to the directory to save Chroma database
CHROMA_PATH = "C:\\Users\\halilibrahim.hatun\\Documents\\Kuika-AI-Hackathon\chroma_db"
def save_to_chroma(chunks: list):
  """
  Save the given list of Document objects to a Chroma database.
  Args:
  chunks (list[Document]): List of Document objects representing text chunks to save.
  Returns:
  None
  """

  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_texts(
    chunks,
    OpenAIEmbeddings(model='text-embedding-3-large',
                    api_key=os.environ['OPENAI_API_KEY']),
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


save_to_chroma(chunks=chunks)

C:\Users\halilibrahim.hatun\Documents\venvs\uniteks_cpu\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Saved 132 chunks to C:\Users\halilibrahim.hatun\Documents\Kuika-AI-Hackathon\chroma_db.


## Inferencing

In [14]:
# YOU MUST - Use same embedding function as before
embedding_function = OpenAIEmbeddings(model='text-embedding-3-large',
                api_key=os.environ['OPENAI_API_KEY'])

# Prepare the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [25]:
def query_rag(query_text):
    """
    Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
    Args:
    - query_text (str): The text to query the RAG system with.
    Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
    """
    
    
    # Retrieving the context from the DB using similarity search
    results = db.similarity_search_with_relevance_scores(query_text, k=5)
    
    # Check if there are any matching results or if the relevance score is too low
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
    

    
    return [doc.page_content for doc, _score in results]


In [26]:
# Let's call our function we have defined
response_text = query_rag("Seyahat prosedüründen bahset")
# and finally, inspect our final response!
print(response_text)

Unable to find matching results.
['Logos Seyahat Prosedürü Sunumu\nAgenda\nGiriş\nKapsam ve Sorumlular\nTanımlar\nSeyahat Onay Süreci\nSeyahat Sebepleri\nSınıflandırma ve Limitler\nUlaşım\nKonaklama ve Yemek\nŞirket Tarafından Karşılanan Masraflar\nŞirket Kredi Kartları\nSeyahat Formu ve Seyahat Avansı\nSeyahat Masraflarının Kapatılması\nGiriş\nKapsam ve Sorumlular: Kapsam\nProsedür\n \nKapsamı\nLogos \nçalışanları\n \niçin\n \ngeçerlidir\nÜst \nyönetim', 'Seyahat Süresi: İşin gerektirdiği süre olup, 15 günü aşmayan seyahatler iş seyahati kapsamında değerlendirilir. 15 günü aşan uzun süreli seyahatler uygulama esasları Genel Müdür onayına tabidir.\nSeyahat İzin Formu: Çalışanın iş seyahatinin amacını açıklamak ve seyahat onayı almak için kullanılan formdur.\n4. Sorumluluklar:\nBu prosedürün uygulanmasından Genel Müdür ve İK Müdürü sorumludur.', 'Tanımlar\nSeyahat Onay Süreci\nYurtiçi Seyahat Onayı\nÖnce Bölüm Müdürü onayı gereklidir\nArdından bir üst yönetici onayı alınır\nYurtdışı Sey